In [1]:
!pip install lightgbm

In [2]:
import pandas as pd
import json
import datetime as dt
import numpy as np
## m

## Load Data

In [3]:
def get_one_hot(df, feature):
    one_hot = pd.get_dummies(df[feature])
    one_hot.columns = [feature+'_'+str(i.lower().translate(str.maketrans('áéíóúü','aeiouu'))) for i in one_hot.columns]
    return one_hot

In [4]:
with open('../data/train.json', mode='r') as f:
    data = json.load(f)
list_dict_data = []
for i in data:
    dict_data = {}
    for j in i:
        dict_data[list(j.keys())[0]] = j[list(j.keys())[0]]
    list_dict_data.append(dict_data)
train = pd.DataFrame(list_dict_data)
train.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(train.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

train.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
train['fecha_publicacion_anuncio'] = train['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['fecha_captura_informacion'] = train['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['interacciones'] = train['interacciones'].astype('int32')
train['duracion_anuncio_total'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days

##CREANDO CAMPO DIFF DIAS JLP
train = train.sort_values(['id_anuncio','fecha_captura_informacion'], ascending = True)
grouped = train.groupby('id_anuncio')
diff = lambda x: x['fecha_captura_informacion'].replace(0, np.NaN).diff() 
train["DIFF"] = grouped.apply(diff).reset_index(0, drop = True).dt.days
train['DIFF'].fillna(train['duracion_anuncio_total'], inplace = True)
##

# train['duracion_anuncio_parcial'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# mask = train['id_anuncio'].astype(str)=='None'
# train.loc[mask, 'id_anuncio'] = train[mask]['id_observacion']
# train['key'] = train['id_anuncio'].astype(str) +'_'+\
#                train['id_conjunto_anuncios'].astype(str) +'_'+\
#                train['id_campania'].astype(str) +'_'+\
#                train['fecha_captura_informacion'].astype(str)
train.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True)
train.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True)
train = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']

list_df = []
for i in var_categ:
    df = get_one_hot(train, i)
    list_df.append(df)
train_categ = pd.concat(list_df,axis=1)

train_final = train.join(train_categ)
train_final.drop(var_categ, axis=1, inplace=True)
Y = train_final['interacciones']
train_final.drop(['interacciones'], axis=1, inplace=True)
train_final

,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,DIFF,fuente_direct,fuente_fb + ig,fuente_facebook,...,formato_video,nombre_campania_aon,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_000129,437.500000,56939.0,42590.0,34748.0,1.139250e+05,3.0,3.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
ID_000126,250.000000,51455.0,42138.0,38094.0,7.094100e+04,0.0,0.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
ID_000127,250.000000,51380.0,41780.0,37794.0,7.208100e+04,1.0,1.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
ID_000128,250.000000,50290.0,40958.0,36977.0,7.106100e+04,2.0,1.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
ID_000125,250.000000,52213.0,42609.0,38686.0,7.240000e+04,3.0,1.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_005561,461.538461,NaN,NaN,NaN,6.593407e+04,NaN,NaN,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ID_005446,272.500000,NaN,NaN,NaN,1.362500e+05,NaN,NaN,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ID_005487,1576.923077,NaN,NaN,NaN,1.051282e+06,NaN,NaN,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
# validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
validation.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(validation.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

validation.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
validation['fecha_publicacion_anuncio'] = validation['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['fecha_captura_informacion'] = validation['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['duracion_anuncio_total'] = (validation['fecha_captura_informacion'] - validation['fecha_publicacion_anuncio']).dt.days

##CREANDO CAMPO DIFF DIAS JLP
validation = validation.sort_values(['id_anuncio','fecha_captura_informacion'], ascending = True)
grouped = validation.groupby('id_anuncio')
diff = lambda x: x['fecha_captura_informacion'].replace(0, np.NaN).diff() 
validation["DIFF"] = grouped.apply(diff).reset_index(0, drop = True).dt.days
validation['DIFF'].fillna(validation['duracion_anuncio_total'], inplace = True)
##

# train['duracion_anuncio_parcial'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# mask = train['id_anuncio'].astype(str)=='None'
# train.loc[mask, 'id_anuncio'] = train[mask]['id_observacion']
# train['key'] = train['id_anuncio'].astype(str) +'_'+\
#                train['id_conjunto_anuncios'].astype(str) +'_'+\
#                train['id_campania'].astype(str) +'_'+\
#                train['fecha_captura_informacion'].astype(str)
validation.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True)
validation.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True)
# validation = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']
list_df = []
for i in var_categ:
    df = get_one_hot(validation, i)
    list_df.append(df)
validation_categ = pd.concat(list_df,axis=1)

validation_final = validation.join(validation_categ)
validation_final.drop(var_categ, axis=1, inplace=True)
validation_final = validation_final.set_index('id_observacion')
validation_final['gasto'] = validation_final['gasto'].replace({'40000%':40000})
validation_final['gasto'] = validation_final['gasto'].astype(float)
validation_final

,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,DIFF,fuente_direct,fuente_fb + ig,fuente_facebook,...,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_gastronomia,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_001302,49.843780,0.0,0.0,0.0,10250.00000,0.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001303,243.361200,42211.0,41183.0,40066.0,45521.00000,2.0,2.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001317,210.701380,35514.0,34674.0,33773.0,37989.00000,3.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001298,75.990030,13436.0,13075.0,12674.0,14713.00000,4.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001304,176.074470,28822.0,28091.0,27341.0,31268.00000,5.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002376,461.538461,NaN,NaN,NaN,65934.06593,NaN,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_002322,272.500000,NaN,NaN,NaN,136250.00000,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_002377,461.538461,NaN,NaN,NaN,65934.06593,NaN,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
features = list(set(train_final.columns).intersection(set(validation_final.columns)))
train_final = train_final[features]
validation_final = validation_final[features]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(train_final, Y, test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4449, 42), (1113, 42), (4449,), (1113,))

In [8]:
from lightgbm import LGBMRegressor
learner = LGBMRegressor(n_estimators=1000)
learner.fit(x_train, y_train,  early_stopping_rounds=10, eval_metric="mae",
                eval_set=[(x_train, y_train), (x_test, y_test)], verbose=50)

[50]	training's l1: 134.517	training's l2: 355905	valid_1's l1: 187.445	valid_1's l2: 379191


C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(n_estimators=1000)

In [9]:
validation_final['INTERACCIONES'] = learner.predict(validation_final)
validation_final

C:\Users\usuario\AppData\Local\Temp/ipykernel_15288/1123091501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_final['INTERACCIONES'] = learner.predict(validation_final)


,nombre_campania_fin de ano,fuente_teads,nombre_campania_aon,formato_rm display,DIFF,formato_key words,formato_banner rm,formato_encuesta,formato_stories,formato_bumper ad,...,objetivo_campania_brand awareness,objetivo_campania_event responses,fuente_facebook,objetivo_campania_engagement,visitas_al_75,formato_video,nombre_campania_equity,nombre_campania_promo cusquena,nombre_campania_gastronomy,INTERACCIONES
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_001302,0,1,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0.0,1,0,0,0,168.894961
ID_V_001303,0,1,0,0,2.0,0,0,0,0,0,...,0,0,0,0,41183.0,1,0,0,0,287.810898
ID_V_001317,0,1,0,0,1.0,0,0,0,0,0,...,0,0,0,0,34674.0,1,0,0,0,264.389791
ID_V_001298,0,1,0,0,1.0,0,0,0,0,0,...,0,0,0,0,13075.0,1,0,0,0,73.244826
ID_V_001304,0,1,0,0,1.0,0,0,0,0,0,...,0,0,0,0,28091.0,1,0,0,0,185.926161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002376,0,0,0,0,NaN,0,0,0,0,0,...,0,0,0,0,NaN,0,0,0,0,180.899573
ID_V_002322,0,0,1,0,NaN,0,0,0,0,0,...,0,0,0,0,NaN,0,0,0,0,13.965850
ID_V_002377,0,0,0,0,NaN,0,0,0,0,0,...,0,0,0,0,NaN,0,0,0,0,180.899573


In [10]:
validation_final = validation_final.reset_index()
validation_final = validation_final[['id_observacion','INTERACCIONES']]
# ID_OBSERVACION, INTERACCIONES.
validation_final.columns = ['ID_OBSERVACION', 'INTERACCIONES']
validation_final

,ID_OBSERVACION,INTERACCIONES
0,ID_V_001302,168.894961
1,ID_V_001303,287.810898
2,ID_V_001317,264.389791
3,ID_V_001298,73.244826
4,ID_V_001304,185.926161
...,...,...
2373,ID_V_002376,180.899573
2374,ID_V_002322,13.965850
2375,ID_V_002377,180.899573
2376,ID_V_002378,180.899573


In [11]:
mask = validation_final['INTERACCIONES']<=0
validation_final.loc[mask,'INTERACCIONES'] = 0

In [12]:
validation_final.to_csv('../results/test4.csv', index=False)